<figure>
  <center><img style='height: 80%; width: 80%; object-fit: contain' src="../images/slot_machine.jpg" /></center>
  <center><figcaption>The k-armed bandit problem is as if you were playing multiple slot machines at once</figcaption></center>
</figure>

In reinforcement learning (RL), the primary distinction between RL and supervised learning (SL) is that RL *evaluates* the actions of the bot rather than *instructing* it. SL simply indicates the correct action to be taken, regardless of the action taken, while RL indicates how good the action was. To study the evaluative aspect of RL, we will consider a simplified setting called the k-armed bandit problem.

In this problem, you are faced repeatedly with a choice among k different actions. After each choice, you receive a numerical reward chosen from a stationary probability density function that depends on the action you selected. Your goal is to maximize the expected total reward over a specified time period, such as 1000 time steps. 

The problem is named after the analogy to a slot machine (the "one-armed bandit"), except that it has k levers. Each action has an expected reward, referred to as its **value**. To estimate this value, we would need a joint distribution of the reward and the action.

$$ q_{*}(a) := \mathbf{E}[R_{t}|A_{t}=a] $$

Where $q_{*}(a)$ is the expected reqward, given that action $a$ is selected. $R_{t}$ is the reward and $A_{t}$ is the action taken at time $t$. The subscript $*$ indicates that this is the true value, which is unknown to the agent but it is the value we are trying to estimate.

One way to estimate the values of actions is to average the rewards seen when a particular action was taken. This is known as an **action-value method**.

$$ Q_{t}(a) := \frac{\text{sum of rewards when $a$ taken prior to t}}{\text{number of times $a$ taken prior to t}} $$

 However, this is not necessarily the best way to estimate the value. The simplest action selection rule is to choose the action with the highest estimated value, referred to as a greedy action. However, it is often beneficial to also explore non-greedy actions by selecting a random action with a small probability, known as an epsilon-greedy method. This ensures that all actions are sampled an infinite number of times, allowing the estimate of the value to converge to the true value.

To compare the performance of greedy and epsilon-greedy methods, we will analyze a set of 2000 randomly generated k-armed bandit problems with k=10. The true values of the actions will be selected from a normal distribution, and the reward for each action will also be a normal distribution with a mean equal to the true value of the action. We will measure the performance of a learning method over 1000 steps, referred to as one run, and repeat this process 2000 times to effectively evaluate different bandit problems.

In the case of noisier rewards, the epsilon-greedy method performed better than the greedy method. However, when the rewards were less noisy, the greedy method performed just as well as the epsilon-greedy method. Overall, the epsilon-greedy method seemed to be the more robust choice, performing well in both noisy and less noisy environments. It is important to carefully consider the trade-off between exploration and exploitation in RL to achieve optimal performance.